In [4]:
# Install necessary Packages
!pip install -Uq langchain-google-genai

In [7]:
# Setup API Key
from google.colab import userdata
GEMINI_API_KEY_1 = userdata.get('Gemini_API_Key')

In [10]:
# Select and configure Model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = userdata.get('Gemini_API_Key')
)

In [12]:
# test if the model is configured correctly
llm.invoke("Hello World")

AIMessage(content='Hello World!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-66b8530b-90fd-4011-afef-403c23aaace6-0', usage_metadata={'input_tokens': 3, 'output_tokens': 4, 'total_tokens': 7, 'input_token_details': {'cache_read': 0}})

In [13]:
from langchain_core.tools import tool
from nltk.sentiment.vader import SentimentIntensityAnalyzer

def analyze_sentiment(text: str) -> str:

    # Initialize the VADER sentiment analyzer
    sid = SentimentIntensityAnalyzer()

    # Get the sentiment scores
    sentiment_score = sid.polarity_scores(text)

    # Classify sentiment based on the compound score
    if sentiment_score['compound'] >= 0.05:
        return "Positive"
    elif sentiment_score['compound'] <= -0.05:
        return "Negative"
    else:
        return "Neutral"

tools = [analyze_sentiment]



In [14]:
llm_with_tools = llm.bind_tools(tools)

In [24]:
from IPython.display import display


text_input = "Coding becomes easier when you start think logically"

ai_msg = llm.invoke(f"Analyze the sentence:\n{text_input}")
display(ai_msg)

AIMessage(content='The sentence "Coding becomes easier when you start think logically" contains a grammatical error and a subtle semantic issue.\n\n* **Grammatical Error:** The verb "think" should be "thinking."  The sentence should read: "Coding becomes easier when you start *thinking* logically."  This is because "start" requires a gerund (verb ending in -ing) to function correctly as part of this kind of phrase.\n\n* **Semantic Issue:** While the sentence conveys a generally true idea – that logical thinking improves coding ability – the phrasing implies a sudden, almost magical shift.  It suggests that the moment one begins thinking logically, coding instantly becomes easier.  In reality, the improvement is gradual and requires practice and application of logical thinking to coding problems.  A more nuanced sentence might be: "Coding becomes easier with the development of logical thinking skills."  or "The more logically you think, the easier coding becomes."\n\n\nIn short, the sen

In [25]:
from langchain_core.messages import HumanMessage, AIMessage
query = f"Analyze the sentiment in the sentence:\n{text_input}"
messages = [HumanMessage(query)]
display(messages)

[HumanMessage(content='Analyze the sentiment in the sentence:\nCoding becomes easier when you start think logically', additional_kwargs={}, response_metadata={})]

In [26]:
ai_msg_tools = llm_with_tools.invoke(messages)
messages.append(ai_msg_tools)
display(messages)

[HumanMessage(content='Analyze the sentiment in the sentence:\nCoding becomes easier when you start think logically', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'analyze_sentiment', 'arguments': '{"text": "Coding becomes easier when you start think logically"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-673658b1-7df5-4671-b186-12c2a7bbea82-0', tool_calls=[{'name': 'analyze_sentiment', 'args': {'text': 'Coding becomes easier when you start think logically'}, 'id': '0d12b5d2-9267-4225-89ce-3fab8f013d4d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 12, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}})]

In [27]:
display(ai_msg_tools)

AIMessage(content='', additional_kwargs={'function_call': {'name': 'analyze_sentiment', 'arguments': '{"text": "Coding becomes easier when you start think logically"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-673658b1-7df5-4671-b186-12c2a7bbea82-0', tool_calls=[{'name': 'analyze_sentiment', 'args': {'text': 'Coding becomes easier when you start think logically'}, 'id': '0d12b5d2-9267-4225-89ce-3fab8f013d4d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 12, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}})

In [28]:
print(ai_msg_tools.tool_calls)

[{'name': 'analyze_sentiment', 'args': {'text': 'Coding becomes easier when you start think logically'}, 'id': '0d12b5d2-9267-4225-89ce-3fab8f013d4d', 'type': 'tool_call'}]


In [30]:
import nltk

# Download the VADER lexicon
nltk.download('vader_lexicon')
tool_call = {
    "name": "analyze_sentiment",  # Function name to be called
    "text" : text_input
}
selected_tool = {
    "analyze_sentiment": analyze_sentiment,
}

# Get the function based on the tool call name (ensure it's converted to lowercase)
function_to_call = selected_tool.get(tool_call["name"].lower())
if function_to_call:
    tool_msg = function_to_call(tool_call["text"])
   # display(tool_msg)
    messages.append(tool_msg)
else:
    print("Tool not found.")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [31]:
display(messages)

[HumanMessage(content='Analyze the sentiment in the sentence:\nCoding becomes easier when you start think logically', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'function_call': {'name': 'analyze_sentiment', 'arguments': '{"text": "Coding becomes easier when you start think logically"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-673658b1-7df5-4671-b186-12c2a7bbea82-0', tool_calls=[{'name': 'analyze_sentiment', 'args': {'text': 'Coding becomes easier when you start think logically'}, 'id': '0d12b5d2-9267-4225-89ce-3fab8f013d4d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 49, 'output_tokens': 12, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}}),
 'Positive',
 'Positive']

In [32]:
response = llm_with_tools.invoke(messages)

In [33]:
from IPython.display import Markdown
Markdown(response.content)
display(tool_msg)

'Positive'